<img src='sharif_logo.png' alt="SUT logo" width=150 height=150 align=left class="saturate" >

<br>
<font face="Times New Roman">
<div dir=ltr align=center>
<font color=0F5298 size=7>
 Deep Learning <br>
<font color=2565AE size=5>
Computer Engineering Department - Spring 2025  <br>
<font color=3C99D size=5>
          Homework 2:  <br>
<font color=696880 size=4>
           
    

# Assignment Overview

In this assignment, you will explore inference scaling techniques in large language models (LLMs) and evaluate their performance using the Math Benchmark. Throughout the notebook, you will learn about several inference methods, including:

- **Chain-of-Thought (CoT):** A method where the model generates intermediate reasoning steps before providing the final answer.
- **Best-of-n Sampling:** An approach that generates multiple candidate responses and selects the best one based on a scoring function.
- **Beam Search:** A technique that expands several possible sequences simultaneously, choosing the most promising ones based on probability.
- **Self-Refinement:** An iterative process where the model revises its output to improve accuracy and coherence.

The **Math Benchmark** is a suite of challenging mathematical problems designed to test the reasoning and problem-solving capabilities of LLMs. The benchmark includes a variety of questions ranging from basic arithmetic and algebra to more advanced topics such as geometry and calculus. For example, you might be asked to solve an equation like `2x + 5 = 15` or compute the derivative of a function, tasks that assess the model's ability to handle both straightforward and complex mathematical queries.

By the end of this assignment, you will have:
- Gained a deeper understanding of inference time scaling methods in LLMs.
- Compared the effectiveness of different inference techniques using a rigorous math evaluation framework.

Let's dive into the notebook and begin exploring how these methods perform on a challenging set of math problems!


## vLLM: Accelerated Inference Engine for LLMs

vLLM is an open-source project designed to optimize the loading and inference of large language models. By leveraging advanced memory management techniques and dynamic batching, vLLM significantly speeds up the inference process, making it easier to deploy and experiment with LLMs even on hardware with limited resources
So we use vLLM to get results faster.

# installing Dependencies

In [ ]:
!pip install vllm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.4/326.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.4/98.4 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 110.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 90.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.

In [ ]:
!pip install transformers accelerate datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: dill
    Found existing installation: dill 0.4.0
    Uninstalling dill-0.4.0:
      Successfully uninstalled dill-0.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
cudf-cu12 25.2.1 requ

In [ ]:
!pip install --upgrade numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 119.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask-cuda 25.2.0 requires numba<0.61.0a0,>=0.59.1, but you have numba 0.61.2 which is incompatible.
ydf 0.11.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 4.25.7 which is incompatible.
cuml-cu12 25.2.1 requires numba<0.61.0a0,>=0.59.1, but you have numba 0.61.2 which is incompatible.
cudf-cu12 25.2.1 requires numba<0.61.0a0,>=0.59.1, but you have numba 0.61.2 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.5 which is incompatible.
distributed-ucxx-cu12 0

In [ ]:
import os
os.kill(os.getpid(), 9)


This command launches a vLLM inference server with:
- Model: `DeepSeek-R1-Distill-Qwen-1.5B`
- Port: `8000` (default API endpoint)
- Precision: `half` (FP16) for memory efficiency
- Max context length: `3192` tokens

**Note:**  
🔹 Ensure you're using a GPU runtime (T4 or better) in Colab  
🔹 Only run the next cell if this one executes successfully


In [ ]:
!vllm serve "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"   --port 8000   --dtype=half   --max-model-len 3192

INFO 05-09 07:05:56 [__init__.py:239] Automatically detected platform cuda.
2025-05-09 07:05:57.297869: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746774357.332539    2733 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746774357.342635    2733 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-09 07:05:57.375402: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
INFO 05-09 07:06:11 [api_server.py:1043] 

* this cell lunches model in background using vllm

In [ ]:
!nohup vllm serve "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B" --port 8000 --dtype=half --max-model-len 5192 &

nohup: appending output to 'nohup.out'


## LLM Query Function

* This Python function sends prompts to a locally-hosted LLM API and returns the generated response
* you can change max_tokens and temperature as you want



In [ ]:
import requests
def get_llm_response(prompt):
    url = "http://localhost:8000/v1/chat/completions"

    payload = {
        "model": "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
        "messages": [
            {
                "role": "user",
                "content": prompt
            }

        ],
    "max_tokens": 500,
    "temperature": 0.6
    }
    response = requests.post(url, json=payload)
    return response.json()['choices'][0]['message']['content'].strip()

# Test response generation
- testing model with some Math benchmark quesions

In [ ]:
# TODO: Generate a response with these Math benchmark quesions
question1 = "How many positive whole-number divisors does 196 have?"
# real answer : 9
question2 = "What is the distance, in units, between the points $(2, -6)$ and $(-4, 3)$? Express your answer in simplest radical form."
# real answer = 3\\sqrt{13}
question3 = "Define\n\\[p = \\sum_{k = 1}^\\infty \\frac{1}{k^2} \\quad \\text{and} \\quad q = \\sum_{k = 1}^\\infty \\frac{1}{k^3}.\\]Find a way to write\n\\[\\sum_{j = 1}^\\infty \\sum_{k = 1}^\\infty \\frac{1}{(j + k)^3}\\]in terms of $p$ and $q.$"
# real answer = p - q
questions = [question1, question2, question3]
for question in questions:
  response = get_llm_response(question)
  print(response)

Alright, so I need to figure out how many positive whole-number divisors 196 has. Hmm, okay, let's start by recalling what a divisor is. A divisor of a number is another number that can divide into it without leaving a remainder. So, for example, the divisors of 6 are 1, 2, 3, and 6 because each of these numbers can divide 6 evenly.

But 196 is a bit larger, so I need a systematic way to find all its divisors. I remember that one way to find the number of divisors is to factorize the number into its prime factors and then use that to determine the number of possible combinations. Let me try that.

First, I need to factorize 196. Let's see, 196 is an even number because it ends with a 6, so it's divisible by 2. Let me divide 196 by 2:

196 ÷ 2 = 98.

Okay, so 2 is a prime factor, and now I have 98 left. 98 is also even, so let's divide by 2 again:

98 ÷ 2 = 49.

Great, so now I have two 2s. So far, the factorization is 2 × 2 × 49, or 2².

Now, 49 is a number I recognize. It's 7 squared,

In [ ]:
# TODO: Generate a response with these Math benchmark quesions
question1 = "How many positive whole-number divisors does 196 have?"
# real answer : 9
question2 = "What is the distance, in units, between the points $(2, -6)$ and $(-4, 3)$? Express your answer in simplest radical form."
# real answer = 3\\sqrt{13}
question3 = "Define\n\\[p = \\sum_{k = 1}^\\infty \\frac{1}{k^2} \\quad \\text{and} \\quad q = \\sum_{k = 1}^\\infty \\frac{1}{k^3}.\\]Find a way to write\n\\[\\sum_{j = 1}^\\infty \\sum_{k = 1}^\\infty \\frac{1}{(j + k)^3}\\]in terms of $p$ and $q.$"
# real answer = p - q
questions = [question1, question2, question3]
for question in questions:
  response = get_llm_response(question)
  print(response)

Okay, so I have this problem: "How many positive whole-number divisors does 196 have?" Hmm, I remember that to find the number of divisors, I need to look at the prime factorization of the number first. Let me think about how to factorize 196.

Alright, 196 is an even number, so it's divisible by 2. Let me divide it by 2: 196 ÷ 2 = 98. Okay, that's still even, so I can divide by 2 again: 98 ÷ 2 = 49. Now, 49 is a familiar number; it's 7 squared. So, 49 ÷ 7 = 7, and then 7 ÷ 7 = 1. 

So, putting that all together, the prime factorization of 196 is 2 squared times 7 squared. In mathematical terms, that's 2² × 7². 

Now, I recall that if a number is expressed in its prime factorization form as N = p₁^a × p₂^b × ... × pₙ^k, then the total number of positive divisors is given by multiplying (a+1)(b+1)...(k+1). So, applying that here, for 196, which is 2² × 7², the exponents are both 2.

So, the number of divisors would be (2+1) × (2+1). Let me compute that: 3 × 3 = 9. 

Wait, let me make su

# Math Benchmark Evaluation

This cell is dedicated to evaluating the performance of inference scaling methods on the Math Benchmark dataset. The process works as follows:

- **Dataset Loading:** It loads the MATH-500 dataset, which contains a set of challenging math problems along with their correct solutions.
- **Answer Extraction:** The `extract_answer` function is used to parse and extract the final answer from the generated responses. This function specifically looks for a LaTeX-style format (using `\boxed{...}`) to reliably pinpoint the answer.
- **Normalization and Comparison:** Before comparing, both the predicted answer and the ground truth are normalized using several functions. These functions handle different mathematical expressions, such as fractions, matrices, and algebraic expressions, ensuring that the comparison is fair and accurate regardless of formatting differences.
- **Evaluation Loop:** For each problem:
  - The ground truth answer is extracted from the provided solution.
  - A response is generated by the LLM using a designated function.
  - The predicted answer is then extracted and compared against the ground truth.
  - The results for each problem, including whether the predicted answer is correct, are saved for later analysis.
- **Results Analysis:** After processing all problems, the cell aggregates the results and prints a summary, including the total number of problems evaluated, the number of correct answers, and the overall accuracy.

This evaluation method ensures that the output of each inference technique (such as Chain-of-Thought, Best-of-n, Beam Search, and Self-Refinement) is consistently measured against the Math Benchmark, without altering the original answers or evaluation logic.

**Note:**  

🔹 you don't need to modify this cell. Only rewrite the evaluation function portion then

🔹 you need to run this cell before evaluating.


In [ ]:
import json
import os
import re
from typing import Dict, Optional, Union
from datasets import load_dataset
from tqdm import tqdm
import torch

# Load the MATH-500 dataset
def load_math500_dataset():
    dataset = load_dataset("HuggingFaceH4/MATH-500")["test"]
    return dataset

# Extract the last boxed answer from text
def extract_answer(response: str) -> Optional[str]:
    if not response:
        return None
    start_idx = response.rfind('\\boxed{')
    if start_idx == -1:
        return None
    brace_count = 1
    pos = start_idx + 7  # length of '\boxed{'
    while pos < len(response) and brace_count > 0:
        if response[pos] == '{':
            brace_count += 1
        elif response[pos] == '}':
            brace_count -= 1
        pos += 1
    if brace_count == 0:
        answer = response[start_idx + 7:pos - 1]
        return answer.strip()
    return None

# Normalization and comparison functions (unchanged from original)
def normalize_number(num_str: str) -> str:
    try:
        cleaned = re.sub(r'[,\$\\]|\s*(?:cm|m|kg|ft|in|lb|oz|ml|L)$|\s*\\text{[^}]+}', '', num_str).strip()
        if cleaned.startswith('.'):
            cleaned = '0' + cleaned
        num = float(cleaned)
        if abs(num) < 1 and '.' in cleaned:
            decimal_places = len(cleaned.split('.')[1])
            format_str = f"{{:.{decimal_places}f}}"
            result = format_str.format(num)
        else:
            result = str(num)
        return result
    except:
        return num_str

def numerically_equal(str1: str, str2: str) -> bool:
    try:
        return abs(float(str1) - float(str2)) < 1e-10
    except:
        return False

def normalize_fraction(fraction_str: str) -> str:
    try:
        fraction_str = fraction_str.replace('\\dfrac', '\\frac')
        fraction_str = ''.join(fraction_str.split())
        fraction_str = re.sub(r'\s*\\text{[^}]+}', '', fraction_str)
        mixed_brace = re.match(r'^\\frac(\d+)\{(\d+)\}$', fraction_str)
        if mixed_brace:
            num, den = mixed_brace.groups()
            return f"\\frac{{{num}}}{{{den}}}"
        no_braces = re.match(r'^\\frac(\d+)(\d+)$', fraction_str)
        if no_braces:
            num, den = no_braces.groups()
            return f"\\frac{{{num}}}{{{den}}}"
        if '/' in fraction_str and not any(c in fraction_str for c in '\\{}'):
            num, den = fraction_str.split('/')
            return f"\\frac{{{num.strip()}}}{{{den.strip()}}}"
        standard = re.match(r'^\\frac\{([^{}]+)\}\{([^{}]+)\}$', fraction_str)
        if standard:
            num, den = standard.groups()
            return f"\\frac{{{num}}}{{{den}}}"
    except:
        return fraction_str

def normalize_matrix_entry(entry: str) -> str:
    entry = ''.join(entry.split())
    if '/' in entry and not any(c in entry for c in '\\{}'):
        if entry.startswith('-'):
            num, den = entry[1:].split('/')
            return f"-{num.strip()}/{den.strip()}"
        else:
            num, den = entry.split('/')
            return f"{num.strip()}/{den.strip()}"
    entry = entry.replace('\\dfrac', '\\frac')
    frac_match = re.match(r'^(-)?\\frac\{(\d+)\}\{(\d+)\}$', entry)
    if frac_match:
        sign, num, den = frac_match.groups()
        sign = sign if sign else ''
        return f"{sign}{num}/{den}"
    return entry

def normalize_matrix(matrix_str: str) -> str:
    try:
        matrix_str = ''.join(matrix_str.split())
        match = re.match(r'^\\begin\{pmatrix\}(.*?)\\end\{pmatrix\}$', matrix_str)
        if not match:
            return matrix_str
        content = match.group(1)
        rows = content.split('\\\\')
        normalized_rows = []
        for row in rows:
            if '&' in row:
                entries = [normalize_matrix_entry(entry) for entry in row.split('&')]
            else:
                entries = [normalize_matrix_entry(row)]
            normalized_rows.append('&'.join(entries))
        result = "\\begin{pmatrix}" + "\\\\".join(normalized_rows) + "\\end{pmatrix}"
        return result
    except:
        return matrix_str

def normalize_algebraic_expression(expr: str) -> str:
    try:
        expr = ''.join(expr.split())
        monomial_match = re.match(r'^(-?\d*\.?\d*)?([a-zA-Z])(?:\^(-?\d+))?$', expr)
        if monomial_match:
            coeff, var, exp = monomial_match.groups()
            coeff = coeff if coeff and coeff not in ['+', '-'] else ('1' if not coeff else '-1')
            exp = exp if exp else '1'
            if coeff == '1' and exp == '1':
                return var
            elif coeff == '1':
                return f"{var}^{exp}"
            elif coeff == '-1' and exp == '1':
                return f"-{var}"
            elif coeff == '-1':
                return f"-{var}^{exp}"
            elif exp == '1':
                return f"{coeff}{var}"
            else:
                return f"{coeff}{var}^{exp}"
        pi_term_match = re.match(r'^(-?\d*\.?\d*)\\?pi$', expr)
        if pi_term_match:
            coeff = pi_term_match.group(1)
            if not coeff or coeff == '-':
                coeff = '-1' if coeff == '-' else '1'
            return f"{coeff}\\pi"
        frac_pi_match = re.match(r'^\\frac{([^{}]+)}{([^{}]+)}\\?pi$', expr)
        if frac_pi_match:
            num, den = frac_pi_match.groups()
            return f"\\frac{{{num}}}{{{den}}}\\pi"
        frac_match = re.match(r'^\\frac{([^{}]+)}{([^{}]+)}$', expr)
        if frac_match:
            num, den = frac_match.groups()
            return f"\\frac{{{num}}}{{{den}}}"
    except:
        return expr.lower()

def normalize_interval_bound(bound: str) -> str:
    if '\\infty' in bound:
        sign = '-' if bound.startswith('-') else ''
        return f"{sign}\\infty"
    return normalize_answer(bound) or bound

def normalize_interval(interval_str: str) -> str:
    try:
        interval_str = ''.join(interval_str.split())
        match = re.match(r'^\\left?([\[\(])(.*?),(.*?)\\right?([\]\)])$', interval_str)
        if not match:
            match = re.match(r'^([\[\(])(.*?),(.*?)([\]\)])$', interval_str)
            if not match:
                return interval_str
        left_bracket, left_bound, right_bound, right_bracket = match.groups()
        norm_left = normalize_interval_bound(left_bound)
        norm_right = normalize_interval_bound(right_bound)
        return f"\\left{left_bracket}{norm_left},{norm_right}\\right{right_bracket}"
    except:
        return interval_str

def normalize_ordered_tuple(tuple_str: str) -> str:
    try:
        tuple_str = tuple_str.replace('\\dfrac', '\\frac')
        tuple_str = tuple_str.replace('\\left', '').replace('\\right', '')
        tuple_str = re.sub(r'\\?\s+', '', tuple_str)
        inner = tuple_str.strip('()')
        parts = inner.split(',')
        normalized_parts = [normalize_answer(part.strip()) for part in parts if normalize_answer(part.strip())]
        return f"({','.join(normalized_parts)})"
    except:
        return None

def normalize_answer(answer: str) -> str:
    if answer is None:
        return ""
    answer = re.sub(r'\\text{[^}]+(?:inches|feet|meters|cm|m|kg|ft|in|lb|oz|ml|L|per|second|minute|hour)[^}]*}', '', answer)
    answer = re.sub(r'(?<!\\)\s+', '', answer)
    ordered_pair_match = re.match(r'^(?:\\left)?\((.*?)(?:\\right)?\)$', answer)
    if ordered_pair_match:
        content = ordered_pair_match.group(1)
        parts = content.split(',')
        normalized_parts = [normalize_answer(part) for part in parts if normalize_answer(part)]
        return f"({','.join(normalized_parts)})"
    answer = ''.join(answer.split())
    if not answer:
        return None
    pm_match = re.match(r'^(.*?)(?:\\pm|-)(.*?)$', answer)
    if pm_match:
        left, right = pm_match.groups()
        norm_left = normalize_answer(left) if left else ""
        norm_right = normalize_answer(right) if right else ""
        if norm_left or norm_right:
            return f"{norm_left}\\pm{norm_right}"
    trig_match = re.match(r'^\\(?:sin|cos|tan|cot|sec|csc)\s*([a-zA-Z])$', answer)
    if trig_match:
        variable = trig_match.group(1)
        func_name = re.match(r'^\\(.*?)(?:\s|$)', answer).group(1)
        return f"\\{func_name}{variable}"
    text_match = re.match(r'^(?:\\text{)?([A-Za-z]+)(?:})?$', answer)
    if text_match:
        return text_match.group(1).lower()
    if (answer.startswith('\\left[') or answer.startswith('\\left(') or
        answer.startswith('[') or answer.startswith('(')) and \
       (answer.endswith('\\right]') or answer.endswith('\\right)') or
        answer.endswith(']') or answer.endswith(')')):
        return normalize_interval(answer)
    if answer.startswith('\\begin{pmatrix}') and answer.endswith('\\end{pmatrix}'):
        return normalize_matrix(answer)
    answer = answer.replace('\\dfrac', '\\frac')
    if '\\frac' in answer or '/' in answer:
        return normalize_fraction(answer)
    neg_sqrt_match = re.match(r'^-\\sqrt\{?(\d+)\}?$', answer)
    if neg_sqrt_match:
        num = neg_sqrt_match.group(1)
        return f"-\\sqrt{{{num}}}"
    sqrt_match = re.match(r'^(\d*)?\\sqrt\{?(\d+)\}?$', answer)
    if sqrt_match:
        coeff, num = sqrt_match.groups()
        coeff = coeff if coeff else '1'
        return f"\\sqrt{{{num}}}" if coeff == '1' else f"{coeff}\\sqrt{{{num}}}"
    sqrt_with_coeff_match = re.match(r'^(\d+)\\sqrt\{?(\d+)\}?$', answer)
    if sqrt_with_coeff_match:
        coeff, num = sqrt_with_coeff_match.groups()
        return f"{coeff}\\sqrt{{{num}}}"
    base_match = re.match(r'^(\d+)(?:_\{?(\d+)\}?|_(\d+))$', answer)
    if base_match:
        number, base1, base2 = base_match.groups()
        base = base1 if base1 else base2
        return f"{number}_{base}"
    percent_match = re.match(r'^(\d+(?:\.\d*)?)\s*\\?%$', answer)
    if percent_match:
        return normalize_number(percent_match.group(1))
    unit_match = re.match(r'^(\d+(?:\.\d*)?)\s*(?:(?:\\[,\s])|,)?\s*(?:\\\\)?(?:\\text{(\w+)}|\\?(?:cm|m|kg|ft|in|lb|oz|ml|L))$', answer)
    if unit_match:
        return normalize_number(unit_match.group(1))
    currency_match = re.match(r'^\\?\$?([\d,]+\.?\d*)$', answer)
    if currency_match:
        return normalize_number(currency_match.group(1))
    if re.match(r'^-?[\d,]+$', answer):
        return normalize_number(answer)
    unit_match = re.match(r'^(-?[\d,]+(?:\.\d*)?)\s*(?:\\(?:mbox|text|hbox|displaystyle)\{[^}]+\})?(?:\^?\d)?$', answer)
    if unit_match:
        return normalize_number(unit_match.group(1))
    mc_match = re.match(r'^\\text{\(?([A-Za-z])\)?}$|^\(?([A-Za-z])\)?$', answer)
    if mc_match:
        return (mc_match.group(1) or mc_match.group(2)).lower()
    degree_match = re.match(r'^(-?[\d,]+(?:\.\d*)?)\s*(?:(?:\^?\\circ)|(?:{\\circ})|(?:°))?$', answer)
    if degree_match:
        return normalize_number(degree_match.group(1))
    answer = re.sub(r'\\text{([^{}]+)}', r'\1', answer)
    try:
        return normalize_algebraic_expression(answer)
    except:
        pass
    answer = answer.replace('\\left', '').replace('\\right', '')
    answer = answer.replace('\\(', '(').replace('\\)', ')')
    answer = answer.replace('\\[', '[').replace('\\]', ']')
    answer = answer.replace('\\{', '{').replace('\\}', '}')
    answer = re.sub(r'\\sqrt\{?(\d+)\}?', r'\\sqrt{\1}', answer)
    answer = re.sub(r'\\sqrt{([^{}]+)}', r'\\sqrt\1', answer)
    if re.match(r'^\d+\\%$', answer) or re.match(r'^\d+$', answer):
        answer = re.sub(r'\\%$', '', answer)
    answer = re.sub(r'\\text{([^{}]+)}', r'\1', answer)
    while len(answer) >= 2 and answer[0] == '{' and answer[-1] == '}':
        if '\\frac' in answer:
            break
        answer = answer[1:-1]
    return answer.lower() if answer else None

def compare_answers(correct_answer: str, predicted_answer: Optional[str]) -> bool:
    if predicted_answer is None:
        return False
    if numerically_equal(correct_answer, predicted_answer):
        return True
    normalized_correct = normalize_answer(correct_answer)
    normalized_predicted = normalize_answer(predicted_answer)
    if not normalized_correct or not normalized_predicted:
        return False
    if normalized_correct == "" and normalized_predicted == "":
        return False
    if ('\\left[' in normalized_correct or '\\left(' in normalized_correct) and \
       ('\\left[' in normalized_predicted or '\\left(' in normalized_predicted):
        return normalized_correct == normalized_predicted
    return normalized_correct == normalized_predicted

# Load existing results
def load_existing_results(filename: str) -> list[Dict]:
    try:
        with open(filename, 'r') as f:
            return json.load(f)
    except FileNotFoundError:
        return []

# Save a single result
def save_result(filename: str, result: Dict):
    results = load_existing_results(filename)
    results.append(result)
    with open(filename, 'w') as f:
        json.dump(results, f, indent=2)

# Analyze and print results
def analyze_results(results: list[Dict]):
    total = len(results)
    correct = sum(1 for r in results if r['is_correct'])
    accuracy = correct / total if total > 0 else 0
    print("\n=== Results Summary ===")
    print(f"Total problems: {total}")
    print(f"Correct answers: {correct}")
    print(f"Accuracy: {accuracy:.2%}")
    print("\n=== Incorrect Problems ===")
    for r in results:
        if not r['is_correct']:
            print(f"Problem {r['index']}:")
            print(f"Expected: {r['correct_answer']}")
            print(f"Predicted: {r['predicted_answer']}")
            print("---")

# Main evaluation function
def evaluate():
    os.makedirs("results", exist_ok=True)
    results_file = "evaluation_results_math500_deepseek.json"
    dataset = load_math500_dataset()
    existing_results = load_existing_results(results_file)
    processed_indexes = {result['index'] for result in existing_results}
    cnt = 0
    t=0
    for idx, item in enumerate(tqdm(dataset, desc="Evaluating problems")):
        if idx in processed_indexes:
            continue
        t += 1
        problem_text = item['problem']
        correct_answer = extract_answer(item['solution'])  # Extract from 'solution', not 'answer'
        response = get_llm_response(problem_text)
        predicted_answer = extract_answer(response)
        is_correct = compare_answers(correct_answer, predicted_answer)
        result = {
            "index": idx,
            "problem": problem_text,
            "response": response,
            "correct_answer": correct_answer,
            "predicted_answer": predicted_answer,
            "is_correct": is_correct
        }
        save_result(results_file, result)
        if is_correct:
          cnt += 1
        print(f"cnt :  {cnt} idx: {t}")
    final_results = load_existing_results(results_file)
    analyze_results(final_results)



# Customizable CoT Prompt Template
* modify cot prompt then evaluate on math benchmark


In [ ]:
# final answer should be in this format: (because of extract_answer function you can change it if you want)
#\\[
#\\boxed{your_answer_here}
#\\]

COT_PROMPT = """
You are a math problem solver. Your task is to solve the given math problem step by step, showing your reasoning process clearly. After solving the problem, present your final answer in the LaTeX format shown below.

Here is the problem:

{}

Please solve this problem step by step, explaining your reasoning at each stage. Make sure to provide your final answer in the specified format.

**Important:**
- Show all your work and reasoning.
- If the problem involves multiple steps, break them down clearly.
- Use appropriate mathematical notation where necessary.
- Ensure your final answer is presented as shown in the final answer format.

**Final Answer Format:**
\[
\boxed{your_answer_here}
\]
"""

* generate response with cot prompt

In [ ]:
import requests

def get_COT_response(problem):
    prompt = COT_PROMPT + "\n" + problem
    url = "http://localhost:8000/v1/chat/completions"

    payload = {
        "model": "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
        "messages": [
            {
                "role": "user",
                "content": prompt
            }

        ],
    "max_tokens": 1900,
    "temperature": 0.3
    }
    response = requests.post(url, json=payload)
    return response.json()['choices'][0]['message']['content'].strip()

# Evaluate CoT
* modify response generation part to evalute this method.

In [ ]:
def evaluate_cot():
    os.makedirs("results", exist_ok=True)
    results_file = "evaluation_results_math500_deepseek_cot.json"
    dataset = load_math500_dataset()
    existing_results = load_existing_results(results_file)
    processed_indexes = {result['index'] for result in existing_results}
    cnt = 0
    for idx, item in enumerate(tqdm(dataset, desc="Evaluating problems")):
        if idx in processed_indexes:
            continue
        if idx >= 30:
          break
        problem_text = item['problem']
        correct_answer = extract_answer(item['solution'])

        # TODO: Generate a response with cot
        response = get_COT_response(problem_text)
        predicted_answer = extract_answer(response)
        ##########################################################
        is_correct = compare_answers(correct_answer, predicted_answer)
        result = {
            "index": idx,
            "problem": problem_text,
            "response": response,
            "correct_answer": correct_answer,
            "predicted_answer": predicted_answer,
            "is_correct": is_correct
        }
        save_result(results_file, result)
        if is_correct:
          cnt += 1
        print(f"corrects :  {cnt} idx: {idx}")
    final_results = load_existing_results(results_file)
    analyze_results(final_results)

In [ ]:
evaluate_cot()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/412 [00:00<?, ?B/s]

test.jsonl:   0%|          | 0.00/447k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/500 [00:00<?, ? examples/s]

Evaluating problems:   0%|          | 1/500 [00:10<1:24:31, 10.16s/it]

...

corrects :  13 idx: 27


Evaluating problems:   6%|▌         | 29/500 [10:50<2:34:48, 19.72s/it]

corrects :  13 idx: 28


Evaluating problems:   6%|▌         | 30/500 [11:06<2:53:56, 22.20s/it]

corrects :  14 idx: 29

=== Results Summary ===
Total problems: 30
Correct answers: 14
Accuracy: 46.67%

=== Incorrect Problems ===
Problem 1:
Expected: p - q
Predicted: None
---
Problem 9:
Expected: 4
Predicted: None
---
Problem 10:
Expected: 2220
Predicted: None
---
Problem 11:
Expected: \frac{3}{56}
Predicted: None
---
Problem 14:
Expected: \sqrt{51}
Predicted: None
---
Problem 15:
Expected: 6 - 5i
Predicted: None
---
Problem 17:
Expected: \pi
Predicted: None
---
Problem 18:
Expected: 28
Predicted: None
---
Problem 19:
Expected: 3
Predicted: None
---
Problem 20:
Expected: 6+9i
Predicted: 6 + 9i
---
Problem 21:
Expected: 13535
Predicted: None
---
Problem 22:
Expected: 5
Predicted: None
---
Problem 23:
Expected: x=5
Predicted: 5
---
Problem 25:
Expected: 1,-2
Predicted: None
---
Problem 26:
Expected: 144
Predicted: None
---
Problem 28:
Expected: -2 + 7i
Predicted: -2 + 7i
---


## Best-of-N

The Best-of-N approach generates several candidate responses for a problem and then selects the one with the highest average token log-likelihood. This ensures that the final answer, formatted within the `\boxed{}` command, is not only correct in presentation but also statistically the most reliable.


In [ ]:
SYSTEM_PROMPT = '''You are solving mathematics problems.

Please think step by step.

Important: Always end your solution with the final answer in this format:

\\[
\\boxed{your_answer_here}
\\]

The entire answer should be contained completely within the \\boxed{} command.'''

def best_of_n_response(problem, N=5):
    best_answer = None
    best_avg_likelihood = float('-inf')
    best_responses = []
    prompt = SYSTEM_PROMPT + "\n" + problem

    for t in range(N):
        # TODO: Generate a response
        url = "http://localhost:8000/v1/chat/completions"
        payload = {
            "model": "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
            "messages": [
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            "max_tokens": 1900,
            "temperature": 0.7,
            "logprobs": True,
            "top_logprobs": 1
        }

        response_data = requests.post(url, json=payload).json()
        response_text = response_data['choices'][0]['message']['content'].strip()

        token_logprobs = []
        try:
            logprobs_data = response_data['choices'][0]['logprobs']['content']
            for token_info in logprobs_data:
                if 'logprob' in token_info:
                    token_logprobs.append(token_info['logprob'])
        except (KeyError, TypeError) as e:
            print(f"Error extracting logprobs: {e}")
            continue

        if token_logprobs:
            avg_likelihood = sum(token_logprobs) / len(token_logprobs)
            extracted_answer = extract_answer(response_text)

            if extracted_answer:
                best_responses.append({
                    'response': response_text,
                    'answer': extracted_answer,
                    'avg_likelihood': avg_likelihood
                })

    answers_grouped = {}
    for resp in best_responses:
        answer = resp['answer']
        if answer not in answers_grouped:
            answers_grouped[answer] = []
        answers_grouped[answer].append(resp)

    best_group_likelihood = float('-inf')
    for answer, responses in answers_grouped.items():
        group_likelihood = sum(r['avg_likelihood'] for r in responses) / len(responses)

        if group_likelihood > best_group_likelihood:
            best_group_likelihood = group_likelihood
            best_answer = answer

    if best_answer is None and best_responses:
        best_responses.sort(key=lambda x: x['avg_likelihood'], reverse=True)
        best_answer = best_responses[0]['answer']

    return best_answer

# Evaluate best of n

* modify response generation part to evalute this method.

In [ ]:
def evaluate_best_of_n():
    os.makedirs("results", exist_ok=True)
    results_file = "evaluation_results_math500_deepseek_best_of_n.json"
    dataset = load_math500_dataset()
    existing_results = load_existing_results(results_file)
    processed_indexes = {result['index'] for result in existing_results}
    cnt = 0
    for idx, item in enumerate(tqdm(dataset, desc="Evaluating problems")):
        if idx in processed_indexes:
            continue
        if idx >= 30:
          break
        problem_text = item['problem']
        correct_answer = extract_answer(item['solution'])
        # TODO: ##########################################################
        response = best_of_n_response(problem_text)
        predicted_answer = response
        ##########################################################
        is_correct = compare_answers(correct_answer, predicted_answer)
        result = {
            "index": idx,
            "problem": problem_text,
            "response": response,
            "correct_answer": correct_answer,
            "predicted_answer": predicted_answer,
            "is_correct": is_correct
        }
        save_result(results_file, result)
        if is_correct:
          cnt += 1
        print(f"corrects :  {cnt} idx: {idx}")
    final_results = load_existing_results(results_file)
    analyze_results(final_results)

In [ ]:
evaluate_best_of_n()

Evaluating problems:   0%|          | 1/500 [01:13<10:09:08, 73.24s/it]

corrects :  1 idx: 0


Evaluating problems:   0%|          | 2/500 [03:57<17:30:51, 126.61s/it]

corrects :  2 idx: 1


Evaluating problems:   1%|          | 3/500 [05:30<15:20:57, 111.18s/it]

...

corrects :  13 idx: 27


Evaluating problems:   6%|▌         | 29/500 [54:50<15:00:37, 114.73s/it]

corrects :  13 idx: 28


Evaluating problems:   6%|▌         | 30/500 [55:33<14:30:20, 111.11s/it]

corrects :  14 idx: 29

=== Results Summary ===
Total problems: 30
Correct answers: 14
Accuracy: 46.67%

=== Incorrect Problems ===
Problem 9:
Expected: 4
Predicted: None
---
Problem 11:
Expected: \frac{3}{56}
Predicted: None
---
Problem 12:
Expected: 284
Predicted: 176
---
Problem 14:
Expected: \sqrt{51}
Predicted: 10
---
Problem 15:
Expected: 6 - 5i
Predicted: None
---
Problem 17:
Expected: \pi
Predicted: None
---
Problem 18:
Expected: 28
Predicted: None
---
Problem 19:
Expected: 3
Predicted: None
---
Problem 20:
Expected: 6+9i
Predicted: 6 + 9i
---
Problem 21:
Expected: 13535
Predicted: None
---
Problem 22:
Expected: 5
Predicted: 12
---
Problem 23:
Expected: x=5
Predicted: 5
---
Problem 24:
Expected: 10
Predicted: 49\%
---
Problem 25:
Expected: 1,-2
Predicted: None
---
Problem 26:
Expected: 144
Predicted: None
---
Problem 28:
Expected: -2 + 7i
Predicted: -2 + 7i
---


## Beam Search

This cell implements a beam search strategy for generating candidate reasoning chains. The method generates multiple continuations at each reasoning step, scoring each candidate based on its average token log-likelihood. By retaining and expanding only the top candidates, the approach efficiently searches for the most promising chain-of-thought that leads to the final answer in the required format.


In [ ]:
def call_qwen_model_raw(prompt, step_num, temperature=0.8):
    """
    Sends a request to the local Qwen endpoint and returns the generated text
    along with the average token log-probability.
    """
    # Build the prompt. We assume the sample already contains the SYSTEM_PROMPT. you can modify max_tokens for different steps
    max_tokens = 300 if step_num < 3 else 800  # More tokens for final steps

    # TODO: Send a request to the Qwen model and get the response
    url = "http://localhost:8000/v1/chat/completions"
    payload = {
        "model": "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ],
        "max_tokens": max_tokens,
        "temperature": temperature,
        "logprobs": True,
        "top_logprobs": 1
    }

    response_data = requests.post(url, json=payload).json()
    output_text = response_data['choices'][0]['message']['content'].strip()

    token_logprobs = []
    try:
        logprobs_data = response_data['choices'][0]['logprobs']['content']
        for token_info in logprobs_data:
            if 'logprob' in token_info:
                token_logprobs.append(token_info['logprob'])
    except (KeyError, TypeError) as e:
        print(f"Error extracting logprobs: {e}")
        token_logprobs = [-5.0]

    avg_token_prob = sum(token_logprobs) / len(token_logprobs) if token_logprobs else -5.0

    return output_text, avg_token_prob, len(token_logprobs)


class BeamCandidate:
    def __init__(self, sequence, cumulative_log_prob, step_scores, finished=False, num_token=0):
        self.sequence = sequence
        self.cumulative_log_prob = cumulative_log_prob
        self.step_scores = step_scores
        self.finished = finished
        self.num_token = num_token

    def __repr__(self):
        return (f"BeamCandidate(score={self.cumulative_log_prob:.3f}, finished={self.finished}, "
                f"sequence={self.sequence})")


def generate_reasoning_steps(context, step_num, top_k):
    """
    For a given candidate reasoning chain (context), generate top_k candidate continuations
    for the current reasoning step (from 1 to 5). Each candidate is verified using the average
    token logprob as a proxy for quality.
    """
    candidates = []
    for i in range(top_k):
        if step_num == 1:
            candidate_prompt = (
                context +
                "\nLet's approach this step by step. First, I'll understand what we're looking for.\n"
            )
        elif step_num == 2:
            candidate_prompt = (
                context +
                "\nNow, I'll set up the mathematical framework to solve this problem.\n"
            )
        elif step_num == 3:
            candidate_prompt = (
                context +
                "\nNext, I'll work through the calculations and solve for the answer.\n"
            )
        else:
            candidate_prompt = (
                context +
                "\nFinally, I'll verify my solution and present the final answer in the boxed format.\n"
            )

        candidate_step, avg_token_prob, num_token = call_qwen_model_raw(
            candidate_prompt, step_num, temperature=0.7 - (step_num * 0.1)  # Reduce temperature as steps progress
        )

        finished = "\\boxed{" in candidate_step

        candidates.append((candidate_step, avg_token_prob, num_token, finished))

    return candidates


def beam_search(init_problem_prompt, beam_width=3, max_steps=3, top_k=2):
    """
    Implements a beam search over reasoning steps.
    """
    prompt = SYSTEM_PROMPT + "\n" + init_problem_prompt
    initial_candidate = BeamCandidate(prompt, 0.0, [], False, 0)
    beams = [initial_candidate]

    for step_num in range(1, max_steps+1):
        new_beams = []

        for candidate in beams:
            if candidate.finished:
                new_beams.append(candidate)
                continue

            step_candidates = generate_reasoning_steps(candidate.sequence, step_num, top_k)
            for (step_text, score, num_token, finished) in step_candidates:
                new_sequence = candidate.sequence + "\n" + step_text
                new_token_count = candidate.num_token + num_token
                new_step_scores = candidate.step_scores + [score]

                if new_token_count > 0:
                    weighted_cumulative = (candidate.cumulative_log_prob * candidate.num_token +
                                        score * num_token) / new_token_count
                else:
                    weighted_cumulative = score

                new_beams.append(BeamCandidate(
                    new_sequence,
                    weighted_cumulative,
                    new_step_scores,
                    finished,
                    new_token_count
                ))

        if not new_beams:
            break

        beams = sorted(new_beams, key=lambda x: x.cumulative_log_prob, reverse=True)[:beam_width]

        if all(beam.finished for beam in beams):
            break

    finished_beams = [beam for beam in beams if beam.finished]

    if finished_beams:
        best_candidate = max(finished_beams, key=lambda x: x.cumulative_log_prob)
    else:
        best_candidate = max(beams, key=lambda x: x.cumulative_log_prob)

    return best_candidate


def run_qwen_beam_search(problem, beam_width, max_steps, top_k, log_level):
    """
    Sets up the sample prompt, performs beam search,
    and extracts the final answer.
    """
    initial_prompt = problem

    best_candidate = beam_search(
        initial_prompt,
        beam_width=beam_width,
        max_steps=max_steps,
        top_k=top_k
    )

    final_answer = extract_answer(best_candidate.sequence)

    if log_level > 0:
        print(f"Best candidate score: {best_candidate.cumulative_log_prob}")
        print(f"Step scores: {best_candidate.step_scores}")
        if log_level > 1:
            print("Full solution:")
            print(best_candidate.sequence)
        print(f"Final answer: {final_answer}")

    return final_answer

# Evaluate beam search
* modify response generation part to evalute this method.

In [ ]:
def evaluate_beam_search():
    os.makedirs("results", exist_ok=True)
    results_file = "evaluation_results_math500_deepseek_beam_search.json"
    dataset = load_math500_dataset()
    existing_results = load_existing_results(results_file)
    processed_indexes = {result['index'] for result in existing_results}
    cnt = 0
    for idx, item in enumerate(tqdm(dataset, desc="Evaluating problems")):
        if idx in processed_indexes :
            continue
        if idx >= 30:
          break
        problem_text = item['problem']
        correct_answer = extract_answer(item['solution'])
        ##########################################################
        # TODO: Generate a response with beam search
        best_candidate = beam_search(
            problem_text,
            beam_width=3,
            max_steps=4,
            top_k=2
        )

        response = best_candidate.sequence
        predicted_answer = predicted_answer = run_qwen_beam_search(
            problem_text,
            beam_width=3,
            max_steps=4,
            top_k=2,
            log_level=1
        )
        ##########################################################
        is_correct = compare_answers(correct_answer, predicted_answer)
        result = {
            "index": idx,
            "problem": problem_text,
            "response": response,
            "correct_answer": correct_answer,
            "predicted_answer": predicted_answer,
            "is_correct": is_correct
        }
        save_result(results_file, result)
        if is_correct:
          cnt += 1
        print(f"corrects :  {cnt} idx: {idx}")
    final_results = load_existing_results(results_file)
    analyze_results(final_results)

In [ ]:
evaluate_beam_search()

Evaluating problems:   0%|          | 1/500 [02:55<24:20:35, 175.62s/it]

Best candidate score: -0.13441986498021763
Step scores: [-0.23351576247562963, -0.1182804329196612, -0.07960281733603992]
Final answer: (3,\dfrac{\pi}{2})
corrects :  1 idx: 0


Evaluating problems:   0%|          | 2/500 [09:48<43:37:05, 315.31s/it]

Best candidate score: -0.0784770426873795
Step scores: [-0.17624951320389906, -0.13279866033544144, -0.08003559627337381, -0.01988320603966713]
Final answer: p - q
corrects :  2 idx: 1


Evaluating problems:   1%|          | 3/500 [13:23<37:12:28, 269.51s/it]

...

Best candidate score: -0.07847813051401838
Step scores: [-0.12072570980836948, -0.06593164591118693, -0.04837566967801871]
Final answer: 78
corrects :  14 idx: 27


Evaluating problems:   6%|▌         | 29/500 [2:16:51<32:05:39, 245.31s/it]

Best candidate score: -0.09375992122383879
Step scores: [-0.13122302265216906, -0.09598738887657722, -0.05934613300325898]
Final answer: -2 + 7i
corrects :  14 idx: 28


Evaluating problems:   6%|▌         | 30/500 [2:20:10<36:36:07, 280.36s/it]

Best candidate score: -0.06157597962550268
Step scores: [-0.13497983413438003, -0.05271973486989737, -0.03715695954229763]
Final answer: 225
corrects :  15 idx: 29

=== Results Summary ===
Total problems: 30
Correct answers: 15
Accuracy: 50.00%

=== Incorrect Problems ===
Problem 4:
Expected: \text{Evelyn}
Predicted: 3.6
---
Problem 7:
Expected: 90^\circ
Predicted: 
---
Problem 9:
Expected: 4
Predicted: 
---
Problem 11:
Expected: \frac{3}{56}
Predicted: 
---
Problem 12:
Expected: 284
Predicted: None
---
Problem 15:
Expected: 6 - 5i
Predicted: 
---
Problem 18:
Expected: 28
Predicted: 
---
Problem 19:
Expected: 3
Predicted: 
---
Problem 20:
Expected: 6+9i
Predicted: 6 + 9i
---
Problem 21:
Expected: 13535
Predicted: 
---
Problem 22:
Expected: 5
Predicted: 
---
Problem 23:
Expected: x=5
Predicted: 5
---
Problem 25:
Expected: 1,-2
Predicted: 
---
Problem 26:
Expected: 144
Predicted: 
---
Problem 28:
Expected: -2 + 7i
Predicted: -2 + 7i
---


## Self-Refinement

This approach begins by generating an initial solution using the given prompt. It then iteratively refines this output by providing the model with targeted feedback and asking it to improve its response. The process continues until the feedback indicates that no further refinement is necessary, ensuring that the final answer—properly formatted within the `\boxed{}` command—is as accurate and well-reasoned as possible.


In [ ]:
SYSTEM_PROMPT = '''You are solving mathematics problems.

Please think step by step.

Important: Always end your solution with the final answer in this format:

\\[
\\boxed{your_answer_here}
\\]

The entire answer should be contained completely within the \\boxed{} command.'''



def generate_content(prompt):
    url = "http://localhost:8000/v1/chat/completions"
    payload = {
        "model": "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ],
        "max_tokens": 1900,
        "temperature": 0.3
    }

    try:
        response = requests.post(url, json=payload)
        response.raise_for_status()
        output_text = response.json()["choices"][0]["message"]["content"].strip()
        return output_text
    except Exception as e:
        print(f"Error calling local API: {e}")
        return f"Error: {str(e)}"


def extract_answer(output):
    boxed_pattern = r"\\boxed\{(.*?)\}"
    match = re.search(boxed_pattern, output, re.DOTALL)

    if match:
        return match.group(1).strip()


def self_refine(problem, max_iter=2):
    prompt = SYSTEM_PROMPT + "\n" + problem

    current_output = generate_content(prompt)
    print(f"Initial solution:\n{current_output}\n")

    for iteration in range(max_iter):
        feedback_prompt = f"""
I need you to analyze the following solution to a math problem and provide feedback.

ORIGINAL PROBLEM:
{problem}

CURRENT SOLUTION:
{current_output}

Please analyze this solution and provide feedback in the following JSON format:
{{
    "is_correct": true/false,
    "has_errors": true/false,
    "error_description": "Description of any errors or issues found",
    "suggestions": "Specific suggestions for improvement",
    "needs_refinement": true/false
}}

Focus on mathematical correctness, logical flow, and whether the solution properly follows the required format with a boxed answer.
"""

        feedback_response = generate_content(feedback_prompt)
        print(f"Feedback (iteration {iteration+1}):\n{feedback_response}\n")

        try:
            json_match = re.search(r'\{[\s\S]*\}', feedback_response)
            if json_match:
                feedback_json = json.loads(json_match.group(0))
            else:
                print("Couldn't parse feedback as JSON, continuing with refinement anyway.")
                feedback_json = {"needs_refinement": True}

            if not feedback_json.get("needs_refinement", True):
                print("No refinement needed. Solution is correct.")
                break

        except Exception as e:
            print(f"Error parsing feedback: {e}")
            feedback_json = {"needs_refinement": True}

        refine_prompt = f"""
I need you to refine your solution to this math problem.

ORIGINAL PROBLEM:
{problem}

YOUR PREVIOUS SOLUTION:
{current_output}

FEEDBACK:
{feedback_response}

Please provide an improved solution that addresses the feedback. Remember to:
1. Think step by step
2. Ensure your calculations are correct
3. Follow the proper format with a final answer in a boxed command
4. Make your reasoning clear and coherent

Your refined solution:
"""

        refined_output = generate_content(refine_prompt)
        print(f"Refined solution (iteration {iteration+1}):\n{refined_output}\n")

        current_output = refined_output

    final_answer = extract_answer(current_output)

    if final_answer:
        print(f"Final answer extracted: {final_answer}")
        return final_answer
    else:
        print("No boxed answer found in the final solution.")
        return "Failed to extract answer in the required format."


# Evaluate Self-Refinement
* modify response generation part to evalute this method.

In [ ]:
def evaluate_self_refiner():
    os.makedirs("results", exist_ok=True)
    results_file = "evaluation_results_math500_deepseek_self_refiner.json"
    dataset = load_math500_dataset()
    existing_results = load_existing_results(results_file)
    processed_indexes = {result['index'] for result in existing_results}
    cnt = 0
    for idx, item in enumerate(tqdm(dataset, desc="Evaluating problems")):
        if idx in processed_indexes :
            continue
        if idx >= 30:
          break
        problem_text = item['problem']
        correct_answer = extract_answer(item['solution'])
        ##########################################################
        # TODO: Generate a response with self_refine
        response = self_refine(problem_text, max_iter=2)
        predicted_answer = response
        ##########################################################
        is_correct = compare_answers(correct_answer, predicted_answer)
        result = {
            "index": idx,
            "problem": problem_text,
            "response": response,
            "correct_answer": correct_answer,
            "predicted_answer": predicted_answer,
            "is_correct": is_correct
        }
        save_result(results_file, result)
        if is_correct:
          cnt += 1
        print(f"corrects :  {cnt} idx: {idx}")
    final_results = load_existing_results(results_file)
    analyze_results(final_results)

In [ ]:
evaluate_self_refiner()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/412 [00:00<?, ?B/s]

test.jsonl:   0%|          | 0.00/447k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/500 [00:00<?, ? examples/s]

Evaluating problems:   0%|          | 0/500 [00:00<?, ?it/s]

...

Refined solution (iteration 2):
Okay, so I need to figure out the profit the math club made from their bake sale. Let me start by understanding the problem again. They sold 54 cookies at three for $1, 20 cupcakes at $2 each, and 35 brownies at $1 each. The baking cost was $15. I need to calculate their profit by subtracting the baking cost from the total revenue.

First, I'll calculate the revenue from each item:

1. **Cookies:**
   - They sold 54 cookies at 3 for $1.
   - Number of sets sold: 54 ÷ 3 = 18 sets.
   - Revenue from cookies: 18 sets × $1 = $18.

2. **Cupcakes:**
   - They sold 20 cupcakes at $2 each.
   - Revenue from cupcakes: 20 × $2 = $40.

3. **Brownies:**
   - They sold 35 brownies at $1 each.
   - Revenue from brownies: 35 × $1 = $35.

Next, I'll sum up all the revenues:
- Total Revenue = $18 (cookies) + $40 (cupcakes) + $35 (brownies) = $93.

Finally, I'll subtract the baking cost from the total revenue to find the profit:
- Profit = Total Revenue - Baking Cost = $9

Evaluating problems:   6%|▌         | 29/500 [35:39<9:49:55, 75.15s/it] 

Feedback (iteration 1):
Okay, so I need to figure out if the solution provided is correct, if there are any errors, and maybe suggest improvements. Let me go through it step by step.

First, the problem is to rotate the complex number 7 + 2i by 90 degrees counter-clockwise around the origin. The solution says to multiply by i, which is correct because multiplying by e^(iπ/2) gives a 90-degree rotation. 

The user then breaks it down by considering the complex number as a vector (7, 2) and applying the rotation matrix [0, -1; 1, 0]. Multiplying this matrix by the vector (7, 2) gives (-2, 7), which is -2 + 7i. This seems correct.

Alternatively, using the formula z' = z * e^(iθ), where θ is 90 degrees or π/2 radians, gives the same result: -2 + 7i. 

I also considered the polar form approach, converting 7 + 2i into polar coordinates, adding 90 degrees to the angle, and then converting back to rectangular form. This also led to the same result, -2 + 7i.

So, all methods seem to agree. The

Evaluating problems:   6%|▌         | 30/500 [36:57<9:39:00, 73.92s/it]

Refined solution (iteration 2):
Okay, so I'm trying to figure out how to solve this problem about forming different battalions from Greek soldiers. Let me start by understanding what the problem is asking.

We have two types of soldiers: upper class and lower class. There are 5 upper class soldiers and 10 lower class soldiers. The battle of Thermopylae requires 4 upper class soldiers and 8 lower class soldiers. The question is asking how many different battalions can be sent.

Hmm, so I think this is a combinatorics problem because we're dealing with selecting soldiers from different groups. I remember that combinations are used when the order doesn't matter, which is the case here since a battalion is just a group of soldiers, not an ordered list. So, I'll use combinations to calculate the number of ways to choose soldiers from each class.

First, I need to calculate the number of ways to choose 4 soldiers from the 5 upper class soldiers. I think the formula for combinations is:

\[
C